# Tool Calling Agents
Let's write a simple tool calling agent searching a playlist on DuckDuckGo.

In [ ]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, HfApiModel

agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=HfApiModel()
)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

## Custom tools
Let's define a tool with ```@tool``` decorator.

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, tool

# function fetching the highest-rated catering services (simulating search query filtering)
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    
    # dummy example list of catering services and ratings
    services =  {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7
    }
    
    # find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service


agent = ToolCallingAgent(tools=[catering_service_tool], model=HfApiModel())
result = agent.run("Can you give me the name of the highest-rated catering service in Gotham City?")

print(result)

Now let's define a tool as subclass of ```Tool``` Python class.

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, Tool

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea.
    """
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The category of the party, e.g., 'casual', 'formal', or 'superhero'."
        }
    }
    
    output_type = "string"
    
    def forward(self, category: str) -> str:
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), 
                          "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")
    
party_theme_tool = SuperheroPartyThemeTool()
agent = ToolCallingAgent(tools=[party_theme_tool], model=HfApiModel())

result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)

print(result)

Let's now share our custom tool to HuggingFace hub.

In [ ]:
party_theme_tool.push_to_hub("pierclgr/party_theme_tool", token="hf_...")  # replace with your token

Let's then import a tool from the hub.

In [ ]:
from smolagents import load_tool, ToolCallingAgent, HfApiModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = ToolCallingAgent(
    tools=[image_generation_tool],
    model=HfApiModel()
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")

Let's also import a HF Space as a tool.

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, Tool

image_generation_tool = Tool.from_hub(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt",
    trust_remote_code=True
)

model = HfApiModel("Qwen/Quen2.5-Coder-32B-Instruct")

agent = ToolCallingAgent(
    tools=[image_generation_tool],
    model=model
)

agent.run("Improve this prompt, then generate an image of it.",
          additional_args={"user_prompt": "A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala"})

## Import LangChain tools
Let's now import a LangChain tool.

In [ ]:
from langchain.agents import load_tools
from smolagents import ToolCallingAgent, HfApiModel, Tool

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])
agent = ToolCallingAgent(tools=[search_tool], model=HfApiModel())
agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")